<a href="https://colab.research.google.com/github/an-andrey/MAIS202_Project/blob/main/code/MoviePredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Pre-Processing

Make sure to install

In [ ]:
#installing surprise package for the built-in datasets
!pip install scikit-surprise

In [ ]:
import pandas as pd
from surprise import Dataset, Reader
import numpy as np

raw_ratings = "https://raw.githubusercontent.com/an-andrey/MAIS202_Project/refs/heads/main/Movielens100k/ratings.csv"
raw_movies = "https://raw.githubusercontent.com/an-andrey/MAIS202_Project/refs/heads/main/Movielens100k/movies.csv"

#importing the Movie Lens dataset
ratings = pd.read_csv(raw_ratings)
movies = pd.read_csv(raw_movies)

#extracting the release year from the title to get it as a separate column
movies["release_year"] = movies["title"].str.extract(r'\((\d{4})\)')

#separating the ratings into 5 chunks in order to do cross validation
ratings_chunks = np.array_split(ratings, 5)

#setting up the reader for surprise to know the range of ratings
reader = Reader(rating_scale=(1, 5))

data_chunks = []
for chunk in ratings_chunks:
    # Creating raw data for each chunk that can be read by surprise separately
    data_chunks.append(Dataset.load_from_df(chunk[['userId', 'movieId', 'rating']], reader))

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


#Prediction Algorithm

In [ ]:
from surprise import KNNWithMeans, Dataset
from surprise.model_selection import GridSearchCV, cross_validate
import joblib
import pickle

# Define the parameter grid for GridSearchCV
sim_options = {
    "name": ["msd"],
    "min_support": [2],
    "user_based": [True],
}

#for now, these were the best options for a score of about 0.90, have to iterate more to find the best hyper-parameters
{'sim_options': {'name': 'msd', 'min_support': 2, 'user_based': True}}

param_grid = {"sim_options": sim_options}

# Perform GridSearchCV on the entire dataset
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

#cv are the amounts of cross validation.
gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse"], cv=3)
gs.fit(data)

# Print the best scores and parameters
print("Best RMSE score:", gs.best_score["rmse"])
print("Best parameters for RMSE:", gs.best_params["rmse"])

# Perform cross-validation on each chunk using the best parameters for RMSE
best_sim_options = gs.best_params["rmse"]["sim_options"]
algo = KNNWithMeans(sim_options=best_sim_options)

for i, chunk in enumerate(data_chunks):
    print(f"Cross-validation results for chunk {i+1}:")
    cross_validate(algo, chunk, measures=["rmse"], cv=5, verbose=True)

trainset = data.build_full_trainset()
algo.fit(trainset)

# Save the trained model
with open('knn_model.pkl', 'wb') as f:
    pickle.dump(algo, f)




Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Best RMSE score: 0.9026460675064927
Best parameters for RMSE: {'sim_options': {'name': 'msd', 'min_support': 2, 'user_based': True}}
Cross-validation results for chunk 1:
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9550  0.9713  0.9930  0.9817  0.9569  0.9716  0.0145  
Fit time          0.01    0.02    0.0

After saving the model under knn_model.pk1. You can run it by running the cell below

In [ ]:
from surprise import Dataset
import pickle

with open('knn_model.pkl', 'rb') as file:
    knn_model = pickle.load(file)

# Make predictions
prediction = knn_model.predict(1, 596)

print(prediction)


user: 1          item: 596        r_ui = None   est = 4.61   {'actual_k': 40, 'was_impossible': False}
